# Bi-LSTM 기반 N/S 이진 분류기 학습 노트북

In [3]:

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight


In [4]:

# 데이터 로드
df = pd.read_csv('mbti_binary_multi_sentence.csv')
df = df.dropna(subset=['text', 'NS'])


In [5]:

# 텍스트 및 이진 라벨 준비
X = df['text']
y = df['NS']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [6]:
# class_weight 계산
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))
print("📊 클래스 가중치:", class_weights_dict)

📊 클래스 가중치: {0: np.float64(1.656782039289055), 1: np.float64(0.7161167717936276)}


In [7]:

# 토큰화 및 시퀀스 패딩
tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

max_len = 300
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len, padding='post')


In [8]:

# Bi-LSTM 이진 분류 모델
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=20000, output_dim=128, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.3, recurrent_dropout=0.3)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
              metrics=['accuracy'])


c:\Users\User\anaconda3\envs\test\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [9]:

# 학습
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=2,
    batch_size=32
)



Epoch 1/2
 52/554 ━━━━━━━━━━━━━━━━━━━━ 1:48 215ms/step - accuracy: 0.6700 - loss: 0.6542

In [ ]:

# 최종 성능 평가
val_loss, val_acc = model.evaluate(X_val_pad, y_val)
print(f"검증 정확도: {val_acc:.4f}, 검증 손실: {val_loss:.4f}")


139/139 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.7051 - loss: 0.5840
검증 정확도: 0.6974, 검증 손실: 0.5912


In [ ]:
import pandas as pd

# NS 라벨 포함된 CSV 불러오기
df = pd.read_csv('mbti_binary_multi_sentence.csv')

# NS 컬럼 결측 제거 (이미 하셨던 것과 동일)
df = df.dropna(subset=['text', 'NS'])

# 비율 확인
print(df['NS'].value_counts(normalize=True))  # 클래스 비율 출력



NS
1    0.698225
0    0.301775
Name: proportion, dtype: float64


In [ ]:
# 토크나이저 저장
import pickle

with open("ns_tokenizer2.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [ ]:
# 모델 저장
model.save("ns_bilstm_model2.h5")